In [ ]:
from __future__ import division

In [ ]:
%matplotlib inline

#### libraries and functions

https://github.com/juhuntenburg/brainsurfacescripts

In [1]:
import gdist
import numpy as np
import pandas as pd
from pqdict import PQDict
from mayavi import mlab
import seaborn as sns
from scipy import stats as stats
import scipy as sp
import matplotlib.pyplot as plt
from vtk_rw import read_vtk, write_vtk
from graphs import graph_from_mesh, dijkstra, shortest_path, sorted_path
from plotting import plot_surf_stat_map

/home/raid3/huntenburg/workspace/try_openblas/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


### Read and convert data

In [2]:
mesh_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/surfs/lowres_rh_d.vtk'
sulc_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/surfs/lowres_rh_sulc.npy'
embed_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/clust/smooth_3/mask_025_5/rh_embed_10.npy'
t1_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/t1/avg_rh_profiles.npy'
path_file = '/scr/ilz3/myelinconnect/all_data_on_simple_surf/path/rh_lateral_path.1D.roi'

#mesh_file = '/SCR/data/ohbm2016/lowres_rh_d.vtk'
#sulc_file = '/SCR/data/ohbm2016/lowres_rh_sulc.npy'
#embed_file = '/SCR/data/ohbm2016/mask_025_5/rh_embed_10.npy'
#t1_file = '/SCR/data/ohbm2016/avg_rh_profiles.npy'

In [5]:
v,f,d=read_vtk(mesh_file)
vertices = v.astype(np.float64)
faces = f.astype(np.int32)
sulc = np.load(sulc_file)
profiles = np.load(t1_file)
embed = np.load(embed_file)
unsorted_path = np.loadtxt(path_file, dtype='int64')[:,0]

In [4]:
G=graph_from_mesh(vertices, faces, edge_length=True)

### Create path and windows

The graph representation of the surface is used to find the shortest path between the chosen source and target node. For each node on the path a window is created containing all other nodes that lie within a given radius and are closer to this node that to any other node on the path. These single node windows are then combined into a larger window with the chosen width that is sliding along the path.

In [7]:
radius=5
halfwidth=3 # if len(path) cannot be divided by 2*halfwidth, some end nodes will be dropped
overlap=0 # in single windows
source=46644
target=59276

In [6]:
#path=shortest_path(G,source,target)
#path=list(np.load('/scr/ilz3/myelinconnect/all_data_on_simple_surf/path/rh_4664_59276.npy'))
path = sorted_path(G, unsorted_path)
#np.save('/scr/ilz3/myelinconnect/all_data_on_simple_surf/path/rh_4664_59276.npy', np.asarray(path))

In [ ]:
sns.set_style('white')
plot=plot_surf_stat_map(vertices, faces, stat_map=embed[:,0],
                        bg_map=sulc, bg_on_stat=True, 
                        elev=200,azim=20, figsize=(11,10))

In [ ]:
source_target = np.zeros_like(embed[:,0])
source_target[path[0]]=-1
source_target[path[-1]]=1
#source_target[np.where(embed[:,0]-np.min(embed[:,0])<0.005)]=1
#source_target[np.where(np.max(embed[:,0])-embed[:,0]<0.005)]=-1

In [ ]:
sns.set_style('white')
plot=plot_surf_stat_map(vertices, faces, stat_map=source_target,
                        threshold=0.5, alpha=0.1, 
                        elev=200,azim=20, figsize=(11,10))

In [ ]:
pathbrain = np.zeros_like(vertices[:,0])
pathbrain[path] = 1

In [ ]:
sns.set_style('white')
plot=plot_surf_stat_map(vertices, faces, stat_map=pathbrain, bg_map=sulc, bg_on_stat=True, elev=200,azim=20, figsize=(11,10))

In [ ]:
# for each vertex show only those vertices in the sparse matrix, that have a distance < radius
# for these give the precise distance
inradius_matrix=gdist.local_gdist_matrix(vertices, faces, max_distance=radius)

In [ ]:
# get only the vertices that are in a distance < radius to any node in path
inradius_path=[]
for pnode in path:
    inradius_path=list(np.unique(inradius_matrix[:,path].indices))

In [ ]:
# make an array with the radius neighbours in columns and the path nodes in rows
path_x_neighbours=np.zeros((len(inradius_path), len(path)))
for pnode in range(len(path)):
    path_x_neighbours[:,pnode]=np.reshape(inradius_matrix[inradius_path,path[pnode]].toarray(),
                                          (len(inradius_path),))
path_x_neighbours[path_x_neighbours==0]=np.nan

In [ ]:
# for each vertex in this list, find the node on path it is closest to
path_x_neighbours_min=np.zeros((len(inradius_path),1))
for nnode in range(len(inradius_path)):
    path_x_neighbours_min[nnode]=np.nanargmin(path_x_neighbours[nnode,:])

In [ ]:
#for each node on the path, extract those neighbour nodes, that fall into its window
windows=[]
for pnode in range(len(path)):
    window=[path[pnode]]
    indices = [i for i, x in enumerate(list(path_x_neighbours_min)) if x == pnode]
    [window.append(inradius_path[y]) for y in indices]
    windows.append(window)

In [ ]:
# combine single windows for sliding
combined_windows=[]
for window in np.arange(start=halfwidth, stop=len(windows)-halfwidth, step=2*halfwidth-overlap):
    combined_window=[]
    for k in range(window-halfwidth,window+halfwidth):
        combined_window+=[k]#windows[k]
    combined_windows.append(combined_window)

Get windows back to surface space

In [ ]:
# construct a full surface for each window, with only the window nodes set to 1
window_surfs=[]
for combined_window in range(len(combined_windows)):
    window_surf=np.zeros((len(vertices),))

    for j in path:
        window_surf[j]=1
        
    for i in combined_windows[combined_window]:
        window_surf[i]=100
    window_surfs.append(window_surf)

In [ ]:
# construct single surface with all windows differently carrying a different number
allwindow_surf=np.zeros((len(vertices),))
for window in range(len(windows)):
        
    for i in windows[window]:
        allwindow_surf[i]=window+2

In [ ]:
sns.set_style('white')
plot=plot_surf_stat_map(vertices, faces, stat_map=allwindow_surf, bg_map=sulc, bg_on_stat=True, 
                        elev=200,azim=20, figsize=(11,10))

In [ ]:
wsize=[]
for window in combined_windows:
    wsize.append(len(window))
sns.set_style('darkgrid')
plt.plot(wsize)
sns.axlabel=('Window', '# Vertices')

In [ ]:
sns.set_style('white')
surf_1 = plot_surf_stat_map(vertices, faces, stat_map=window_surfs[0], elev=200, azim=20, 
                            threshold=0.5, alpha=0.1, 
                            #cmap='Spectral', 
                            figsize=(11,10))

In [ ]:
sns.set_style('white')
surf_1 = plot_surf_stat_map(vertices, faces, stat_map=window_surfs[20], elev=200, azim=20, 
                            threshold=0.5, alpha=0.1, 
                            #cmap='Spectral', 
                            figsize=(11,10))

In [ ]:
sns.set_style('white')
surf_1 = plot_surf_stat_map(vertices, faces, stat_map=window_surfs[30], elev=200, azim=20, 
                            threshold=0.5, alpha=0.1, 
                            #cmap='Spectral', 
                            figsize=(11,10))

In [ ]:
sns.set_style('white')
surf_1 = plot_surf_stat_map(vertices, faces, stat_map=window_surfs[40], elev=200, azim=20, 
                            threshold=0.5, alpha=0.1, 
                            #cmap='Spectral', 
                            figsize=(11,10))

In [ ]:
sns.set_style('white')
surf_1 = plot_surf_stat_map(vertices, faces, stat_map=window_surfs[-1], elev=200, azim=20, 
                            threshold=0.5, alpha=0.1, 
                            #cmap='Spectral', 
                            figsize=(11,10))

### Sample profiles in windows

In [ ]:
profiles = profiles[:,3:8]

In [ ]:
# sample t1 per window, calc mean profile and sdv
profile_windows=[]
profile_avg=[]
profile_sdv=[]
for window in combined_windows:
    profile_windows.append(profiles[window])
    profile_avg.append(np.mean(profiles[window], axis=0))
    profile_sdv.append(np.std(profiles[window], axis=0))

In [ ]:
sns.set_style('darkgrid')

In [ ]:
colors = sns.color_palette('cubehelix', 10)

In [ ]:
fig = plt.figure(figsize=(15,30))
for i in range(len(profile_avg)):
    ax = fig.add_subplot(int(np.ceil(len(profile_avg)/4)),4,i+1)
    plt.plot(profile_avg[i], label=str(i+1), color=colors[3])
    plt.legend(loc=2)

In [ ]:
fig = plt.figure(figsize=(15,30))
for i in range(len(profile_avg)):
    ax = fig.add_subplot(int(np.ceil(len(profile_avg)/4)),4,i+1)
    plt.plot(profile_avg[i], label=str(i+1), color=colors[3])
    ax.set_ylim([1500,2500])

In [ ]:
fig = plt.figure(figsize=(15,30))
for i in range(len(profile_avg)):
    ax = fig.add_subplot(int(np.ceil(len(profile_avg)/4)),4,i+1)
    for pro in range(len(profile_windows[i])):
        plt.plot(profile_windows[i][pro],color='gray', alpha=0.3)
        plt.plot(profile_avg[i], color='red', linewidth=0.1)

In [ ]:
etamap=np.array(sns.diverging_palette(220, 20, n=len(profile_avg), center='dark'))
fig = plt.figure(figsize=(16,20))
fig.add_subplot(221)
for i in range(len(profile_avg)):
    r=etamap[i][0]
    g=etamap[i][1]
    b=etamap[i][2]
    if i==0:
        plt.plot(profile_avg[i], color=(r,g,b),label='source')
    elif i==len(profile_avg)-1:
        plt.plot(profile_avg[i], color=(r,g,b),label='target')
    else:
        plt.plot(profile_avg[i], color=(r,g,b))
plt.legend(loc=4, fontsize=14)


fig.add_subplot(222)
for i in range(len(profile_avg)):
    r=etamap[i][0]
    g=etamap[i][1]
    b=etamap[i][2]
    if i==0:
        plt.plot(profile_avg[i]-np.mean(profile_avg[i]), color=(r,g,b),label='source', alpha=0.7)
    elif i==len(profile_avg)-1:
        plt.plot(profile_avg[i]-np.mean(profile_avg[i]), color=(r,g,b),label='target', alpha=0.7)
    else:
        plt.plot(profile_avg[i]-np.mean(profile_avg[i]), color=(r,g,b), alpha=0.7)
plt.legend(loc=4, fontsize=14)


fig.add_subplot(223)
for i in range(len(profile_avg)):
    r=etamap[i][0]
    g=etamap[i][1]
    b=etamap[i][2]
    if i==0:
        plt.plot(profile_avg[i]-np.mean(profile_avg, axis=0), color=(r,g,b),label='source')
    elif i==len(profile_avg)-1:
        plt.plot(profile_avg[i]-np.mean(profile_avg, axis=0), color=(r,g,b),label='target')
    else:
        plt.plot(profile_avg[i]-np.mean(profile_avg, axis=0), color=(r,g,b))
plt.legend(loc=4, fontsize=14)

### euclidian distance between chebychev coefficients

In [ ]:
euclid_list=[]
for w in range(len(combined_windows)-1):
    diff=sp.spatial.distance.euclidean(cheb_coeffs[w],cheb_coeffs[w+1])
    euclid_list.append(diff)

In [ ]:
sns.set_context('notebook', font_scale=1.5)
fig = plt.figure(figsize=(15,10))

euclidplot=fig.add_subplot(2,1,1)
plt.plot(euclid_list)
euclidplot.set_xlim([0, len(windows)])
euclidplot.set_xticks(range(0, len(combined_windows), 2))
sns.axlabel('','euclidian distance \nchebychev coefficients', fontsize=20)

corr=fig.add_subplot(2,1,2)
plt.plot(r_corr_avgs)
corr.set_xlim([0,len(windows)])
corr.set_xticks(range(0, len(combined_windows), 2))
sns.axlabel('','corr', fontsize=20)

In [ ]:
def euclid(coeffs):
    euclid_array=np.zeros((coeffs.shape[0], coeffs.shape[0]))
    for i in range(coeffs.shape[0]):
        for j in range(coeffs.shape[0]):
            euclid_array[i,j]=sp.spatial.distance.euclidean(cheb_coeffs[i],cheb_coeffs[j])
    return euclid_array

In [ ]:
euclid_windows=euclid(cheb_coeffs)